### The Assgnment Optimization Problem

* The **assignment problem** deals with optimally assigning or pairing of objects of two distinct sets (e.g. workers and jobs) where each pairing is associated with some cost or benefit. 
* This problem is a **network problem**, but it can also be formulated as an **integer linear problem** (i.e. the solutions must be integer). 

* **Coefficient Matrix ($ c_{i,j} \ ) \ \Leftrightarrow $ Weighted Bipartite Graph**

<img src="AssignmentProblem.png"> 

* Furthermore, it can be proven that a network problem with integer constraint data (e.g. supplies, demands, capacities), has integer optimal solutions! 
* Therefore, the assignment optimization problem can be formulated as a linear problem (i.e. dropping the integrality requirement) because we know the optimal solution will be integer. 

#### General Formulation of the Linear Assignment Problem


$ \ \  \ \ \ \ \ \min / \max f(\overrightarrow{X}) = \sum_i \sum_j c_{i,j}*x_{i,j}$


$s.t. \ \sum_j X_{i,j}=1 \ \forall \ i \  (Each \ staff \ is \ assigned \ to \ one \ role)$


$ \ \ \ \ \ \ \ \sum_i X_{i,j}=1 \ \forall \ j \ (Each \ role \ is \ assigned \ to \ one \ task) $


$ \ \ \ \ \ \ \ \ x_{i,j} \ \in \{0,1\} \ \ \ \ OR \ \ \ \ x_{i,j} \ \geq 0 \ \ when \ formulated \ as \ a \ linear \ problem $

* The assignment problem can also be solved even more efficiently with the **Hungarian algorithm**, a combinatorial optimization algorithm, which takes advantages of the specific charasteristics of the problem. 
* The Hungarian Algorithm is available in the Python module `SciPy` in the sub-module `optimize`, named the `linear_sum_assignment` method (reference [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html)).
* There only data we need to provide to the method is the coefficient matrix.

In [2]:
import numpy as np 
import pandas as pd
import scipy.optimize as opt
import altair as alt

In [3]:
Coefficients = pd.read_excel('TheAssignmentProblem.xlsx', sheet_name='Coefficients')

In [4]:
Coefficients.head(2)

,Coefficients,Role 01,Role 02,Role 03,Role 04,Role 05,Role 06,Role 07,Role 08,Role 09,Role 10,Role 11,Role 12,Role 13,Role 14
0,Person 01,4,12,11,7,1,13,6,2,10,14,5,8,3,9
1,Person 02,11,2,13,4,7,14,6,3,5,8,10,1,12,9


* Setting the "Coefficients" column as the row index for the data frame

In [5]:
Coefficients.set_index("Coefficients", inplace=True)
Coefficients = Coefficients

In [6]:
Coefficients.head()

,Role 01,Role 02,Role 03,Role 04,Role 05,Role 06,Role 07,Role 08,Role 09,Role 10,Role 11,Role 12,Role 13,Role 14
Coefficients,,,,,,,,,,,,,,
Person 01,4,12,11,7,1,13,6,2,10,14,5,8,3,9
Person 02,11,2,13,4,7,14,6,3,5,8,10,1,12,9
Person 03,11,1,6,5,8,4,12,10,2,3,13,14,7,9
Person 04,6,8,9,11,13,10,4,12,3,5,14,1,7,2
Person 05,5,11,13,6,2,8,7,3,10,1,12,4,9,14


* Convert Coefficients Dataframe to **`Numpy`** Matrix to use in **`SciPy.Optimize`**

In [7]:
C = Coefficients.values

* Now we can solve the assignment problem using the **Hungarian Algorithm** through the **`linear_sum_assignment`** method

* **`row_ind`** and **`col_ind`** are the corresponding indeces of the optimal solution that are returned by the method

In [8]:
row_ind, col_ind = opt.linear_sum_assignment(cost_matrix=C,maximize=False)

* Since indices start at zero, we add one to the `row_ind` and `col_ind` respectively
* We can also use the `row_ind` and `col_ind` indices to get the coefficient matrix values of the optimal allocation
* We use these to create a table with the optimal allocation and preference, as well as return the value of the optimal solution

In [9]:
Solution = pd.concat([pd.DataFrame(data=row_ind, columns=["Staff"]) + 1, 
                      pd.DataFrame(data=col_ind, columns=["Roles"]) + 1, 
                      pd.DataFrame(data=C[row_ind, col_ind], columns=["Preference"])], axis=1)

In [10]:
print("Optimal Allocation: \n", Solution, "\n")
print("Objective Value:", C[row_ind, col_ind].sum())
print("Maximum Preference:", Solution["Preference"].max())

Optimal Allocation: 
     Staff  Roles  Preference
0       1      5           1
1       2     12           1
2       3      2           1
3       4     14           2
4       5     10           1
5       6      1           1
6       7      4           1
7       8      3           1
8       9      6           3
9      10      7           1
10     11      9           2
11     12     11           2
12     13     13           2
13     14      8           2 

Objective Value: 21
Maximum Preference: 3


* We print the frequency of the preferences of the optimal allocation to see if any allocations are significantly different than the rest

In [14]:
PrefBars = alt.Chart(Solution).mark_bar().encode(
    x=alt.X("Preference:Q", bin=alt.Bin(step=1, extent=[1, 14]), title="Preference"), 
    y=alt.Y("count(Preference):Q", axis=None, title="Count")
).properties(width=310)
    
PrefText = PrefBars.mark_text(align = "left", baseline = "middle", dy=-7, dx=-2).encode(
    text="count(Preference):Q"
)
    
PrefGraph = PrefBars + PrefText

In [15]:
PrefGraph

alt.LayerChart(...)